In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import dask
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cmcrameri import cm
import babet as bb
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import metpy.calc as mpcalc 
from metpy.units import units

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)
inidates = ['2023-10-11', '2023-10-13', '2023-10-15', '2023-10-17']
experiments = ['pi', 'curr', 'incr']

dask.config.set(**{'array.slicing.split_large_chunks': True})

# Import data

In [ ]:
# Import forecast data 
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/pl/pf' # TODO: change as needed, data directory
exp = {}
for experiment in experiments:
    exp[experiment] = xr.open_mfdataset(os.path.join(base_dir.format(experiment), '*.nc'), preprocess=bb.Data.preproc_ds)

In [ ]:
# Import forecast data at surface
base_dir = '/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/MED-R/EXP/{}/EU025/sfc/pf' # TODO: change as needed, data directory
exp_sfc = {}
for experiment in experiments:
    exp_sfc[experiment] = xr.open_mfdataset(os.path.join(base_dir.format(experiment), '*.nc'), preprocess=bb.Data.preproc_ds)

In [ ]:
# Plot settings

# UK
lat_max = 60
lat_min = 42
lon_min = -12
lon_max = 5

# #Europe
# lat_max = 70
# lat_min = 33
# lon_min = -27
# lon_max = 25

euroatlantic = [lon_min-13, lon_max, lat_min-5, lat_max+6]
uk = [-11, 10, 48, 70]
northsea = [-17, 20, 40, 70]

# Calculate vertically aeraged vertical velocity and vertically integarted specific humidity


$$\langle w \rangle = \frac{\int w \cdot \Delta P}{\int \Delta P}$$

In [ ]:
def calc_average_w(ds, upper=250):
    
    ds = ds.copy(deep=True).sel(level=slice(upper, 1000))
    
    # Compute the pressure thickness (ΔP) between levels
    # The last dimension should be the pressure dimension
    delta_p = ds.level.diff('level').rename('delta_p')

    # Align ΔP with the original dataset (it will be one less in size)
    delta_p = delta_p.assign_coords(p=ds['level'].isel(level=slice(1, None)))

    # Compute the mass-weighted vertical velocity
    weighted_w = ds['w'].isel(level=slice(1, None)) * delta_p

    # Compute the mass-weighted average
    mass_weighted_average = (weighted_w.sum(dim='level') / delta_p.sum(dim='level')).rename('mass_weighted_w')

    # Add the result to your dataset
    ds['mass_weighted_w'] = mass_weighted_average
    
    return ds


In [ ]:
def calc_qs(ds):

    # Constants
    epsilon = 0.622  # Ratio of gas constants (R_v / R_d)
    R_d = 287.05  # Gas constant for dry air (J/kg/K)
    R_v = 461.5  # Gas constant for water vapor (J/kg/K)

    # Assume your dataset `ds` contains:
    # - 'T': temperature (Kelvin)
    # - 'p': pressure (Pa)

    # Convert temperature from Kelvin to Celsius
    T_c = ds['t'] - 273.15

    # Compute saturation vapor pressure (e_s) in hPa
    e_s_hpa = 6.112 * np.exp((17.67 * T_c) / (T_c + 243.5))

    # Convert saturation vapor pressure to Pa
    e_s = e_s_hpa * 100  # Convert hPa to Pa

    # Compute saturation specific humidity (q_s)
    q_s = (epsilon * e_s) / (ds['level'] - (1 - epsilon) * e_s)

    # Add q_s to the dataset
    ds['q_s'] = q_s
    
    return ds

In [ ]:
def integate_qs(ds, upper=250):
    # Constants
    g = 9.81  # gravitational acceleration in m/s^2

    # Assume `ds` is your xarray Dataset with:
    # - 'q_s': saturation specific humidity (kg/kg)
    # - 'p': pressure levels (Pa), sorted from top to bottom

    # Calculate saturation specific humidity at each level
    ds = calc_qs(ds).copy(deep=True)

    # Compute pressure thickness (Δp) for each layer
    delta_p = ds['level'].diff('level').rename('delta_p')

    # Align Δp to match dimensions of q_s (it will have one fewer element)
    delta_p = delta_p.assign_coords(p=ds['level'].isel(level=slice(1, None)))

    # Compute the weighted saturation specific humidity (q_s * Δp)
    weighted_qs = ds['q_s'].isel(level=slice(1, None)) * delta_p

    # Integrate vertically by summing over the pressure dimension
    vertically_integrated_qs = (weighted_qs.sum(dim='level') / g).rename('Q_s')

    # Add the result to the dataset for further use
    ds['Q_s'] = vertically_integrated_qs

    return ds

I am saving the data for the plots here directly after calculating them to speed up the plotting

In [ ]:
# plot Q_s and w changes  # TODO: comment out after using once
starttime = '2023-10-19 00'
endtime = '2023-10-22 00'
exp = {exp_key: integate_qs(calc_average_w(exp[exp_key])) for exp_key in exp.keys()}
Q_s_dict = {exp_key: {ini_key: (exp[exp_key]-exp['curr']).Q_s.sel(inidate=ini_key, time=slice(starttime, endtime)).mean(dim=['time', 'number']).squeeze() for ini_key in ['2023-10-15', '2023-10-17']} for exp_key in exp.keys()}
w_dict = {exp_key: {ini_key: (exp[exp_key]-exp['curr']).mass_weighted_w.sel(inidate=ini_key, time=slice(starttime, endtime)).mean(dim=['time', 'number']).squeeze() for ini_key in ['2023-10-15', '2023-10-17']} for exp_key in exp.keys()}

# save data for plot
np.save('../data/A05_Q_s_vals.npy', Q_s_dict)
np.save('../data/A05_w_vals.npy', w_dict)

In [ ]:
# Load data for plot
Q_s_dict = np.load('../data/A05_Q_s_vals.npy', allow_pickle=True).item()
w_dict = np.load('../data/A05_w_vals.npy', allow_pickle=True).item()

# Plot 

In [ ]:
# plot Q_s and w changes for pi 
exp = {exp_key: integate_qs(calc_average_w(exp[exp_key])) for exp_key in exp.keys()}

inidates = ['2023-10-15', '2023-10-17']
starttime = '2023-10-19 00'
endtime = '2023-10-22 00'

fig = plt.figure(1, figsize=(15, 10))
projection = ccrs.PlateCarree()
fs = 15

w_min = -0.1
w_max = 0.1
clevs_w = np.linspace(w_min, w_max, 17)

qs_min = -10
qs_max = 10
clevs_qs = np.linspace(qs_min, qs_max, 17)


for i, inidate in enumerate(inidates):
    ax1 = plt.subplot(2, 2, 2*i+1, projection=projection)
    c_qs = ax1.contourf(exp['pi'].longitude, exp['pi'].latitude, Q_s_dict['pi'][inidate], # TODO: change forcing experiment as needed, pi here
                        clevs_qs, cmap=cm.broc_r, 
                        transform=projection, 
                        extend='both')
    ax1.set_title(rf'ini {inidate}, vertically int. $q_s$')
    ax1.set_extent(uk, projection)
    ax1.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'gray', zorder = 14)
    rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                edgecolor='k', 
                                facecolor='none',
                                transform=projection)
    ax1.add_patch(rectangle)
    rectangle.set_zorder(17)

    ax2 = plt.subplot(2, 2, 2*i+2, projection=projection)
    c_w = ax2.contourf(exp['pi'].longitude, exp['pi'].latitude, w_dict['pi'][inidate], # TODO: change forcing experiment as needed, pi here
                       clevs_w, cmap=cm.bam, 
                       transform=projection, 
                       extend='both')
    ax2.set_title(rf'ini {inidate}, vertically averaged $w$')
    ax2.set_extent(uk, projection)
    ax2.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'gray', zorder = 14)
    rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                edgecolor='k', 
                                facecolor='none',
                                transform=projection)
    ax2.add_patch(rectangle)
    rectangle.set_zorder(17)

# colourbars
cax = ax2.inset_axes([1.1, 0.02, 0.07, 0.9])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(c_w, cax=cax, label=rf'difference in vertically averaged $w$', extend = 'both', shrink=0.8)
cbar.set_label(label=rf'difference in vertically averaged $w$', size=fs-3)
cbar.ax.tick_params(labelsize=fs-3)

cax = ax2.inset_axes([1.5, 0.02, 0.07, 0.9])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(c_qs, cax=cax, label=rf'difference in vertically int. $q_s$', extend = 'both', shrink=0.8)
cbar.set_label(label=rf'difference in vertically int. $q_s$', size=fs-3)
cbar.ax.tick_params(labelsize=fs-3)

plt.tight_layout()

In [ ]:
# plot Q_s and w changes for incr
exp = {exp_key: integate_qs(calc_average_w(exp[exp_key])) for exp_key in exp.keys()}

inidates = ['2023-10-15', '2023-10-17']
starttime = '2023-10-19 00'
endtime = '2023-10-22 00'

fig = plt.figure(1, figsize=(15, 10))
projection = ccrs.PlateCarree()
fs = 15

w_min = -0.1
w_max = 0.1
clevs_w = np.linspace(w_min, w_max, 17)

qs_min = -10
qs_max = 10
clevs_qs = np.linspace(qs_min, qs_max, 17)


for i, inidate in enumerate(inidates):
    ax1 = plt.subplot(2, 2, 2*i+1, projection=projection)
    c_qs = ax1.contourf(exp['incr'].longitude, exp['incr'].latitude, Q_s_dict['incr'][inidate], # TODO: change forcing experiment as needed, incr/future here
                        clevs_qs, cmap=cm.broc_r, 
                        transform=projection, 
                        extend='both')
    ax1.set_title(rf'ini {inidate}, vertically int. $q_s$')
    ax1.set_extent(uk, projection)
    ax1.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'gray', zorder = 14)
    rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                edgecolor='k', 
                                facecolor='none',
                                transform=projection)
    ax1.add_patch(rectangle)
    rectangle.set_zorder(17)

    ax2 = plt.subplot(2, 2, 2*i+2, projection=projection)
    c_w = ax2.contourf(exp['incr'].longitude, exp['incr'].latitude, w_dict['incr'][inidate], # TODO: change forcing experiment as needed, incr/future here
                       clevs_w, cmap=cm.bam, 
                       transform=projection, 
                       extend='both')
    ax2.set_title(rf'ini {inidate}, vertically averaged $w$')
    ax2.set_extent(uk, projection)
    ax2.add_feature(cfeature.COASTLINE.with_scale('50m'), color = 'gray', zorder = 14)
    rectangle = patches.Rectangle((-4, 55.5), 2, 2, linewidth=2, 
                                edgecolor='k', 
                                facecolor='none',
                                transform=projection)
    ax2.add_patch(rectangle)
    rectangle.set_zorder(17)

# colourbars
cax = ax2.inset_axes([1.1, 0.02, 0.07, 0.9])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(c_w, cax=cax, label=rf'difference in vertically averaged $w$', extend = 'both', shrink=0.8)
cbar.set_label(label=rf'difference in vertically averaged $w$', size=fs-3)
cbar.ax.tick_params(labelsize=fs-3)

cax = ax2.inset_axes([1.5, 0.02, 0.07, 0.9])  # creates inset, [x0,y0, width, height]
cbar = fig.colorbar(c_qs, cax=cax, label=rf'difference in vertically int. $q_s$', extend = 'both', shrink=0.8)
cbar.set_label(label=rf'difference in vertically int. $q_s$', size=fs-3)
cbar.ax.tick_params(labelsize=fs-3)

plt.tight_layout()